c: confidence of X=>Y := |{t| X & Y in t}| / |{t | X in t}|

s: support of    X=>Y := |{t| XUY in t}| / T

L_k = set of largest k-itemsets (itemset, support)
<br/>
C_k = set of candiadte k-itemsets (itemset, support)
<br/>
$\bar{C}_k = set of candidate k-itemsets when TIDs of the generatig transactions are kept associated with the candiadtes

In [ ]:
transactions = [
    ["milk", "bread", "meat", "beer", "coffee"],
    ["rice", "mean"],
    ["beer", "milk", "sugar", "rice"]
]

In [ ]:
L_3 = [
    [1, 2, 3], 
    [1, 2, 4], 
    [1, 3, 4], 
    [1, 3, 5],
    [2, 3, 4]
]

Let's first define our itemset object. A class k_itemset is implemented to capture an itemset which will be a set of items and support 
of that set. <br/>Furthur we implement repr method to for logging the itemset object, <br/> __eq__ method for being able to create non-repeating combinations and also check 
if a set of items is in a list of itemset objects, __lt__ for sorting list of itemsets.

In [1]:
from functools import total_ordering


@total_ordering
class k_itemset:
    def __init__(self, itemset: set, support=0):
        self.itemset = itemset
        self.support = support
    
    def __repr__(self):
        return f"k_itemset(itemset={self.itemset}, support={self.support})"
    
    def __eq__(self, other):
        try:
            return self.itemset == other.itemset
        except AttributeError:
            return self.itemset == other
    
    def __lt__(self, other):
        return self.support < other.support
    
    def __hash__(self):
        return hash(tuple(self.itemset))

We transform raw transactions, which is a list of list of item names, to a list of set of items with indexes, <br/>
and items_map which is a dictionary with item_name as key and item_index as index

In [5]:
transactions = [
    ["milk", "bread", "meat"],
    ["rice", "bread", "beer"],
    ["milk", "rice", "bread", "beer"],
    ["rice", "beer"]
]
def transform_transactions(transactions):
    items = set()
    for trx in transactions:
        trx = set(map(lambda t: t.lower(), trx))
        items = items.union(trx)
    items_map = {item: ind for ind, item in enumerate(sorted(items), 1)}
    trxs = [set(map(lambda t: items_map[t.lower()], trx)) for trx in transactions]
    return items_map, trxs

items_map, transactions = transform_transactions(transactions)
print("items_map => ", items_map)
print("transactions encoded => ", transactions)

items_map =>  {'beer': 1, 'bread': 2, 'meat': 3, 'milk': 4, 'rice': 5}
transactions encoded =>  [{2, 3, 4}, {1, 2, 5}, {1, 2, 4, 5}, {1, 5}]


==============================================================================================

Generating candiate list for the first time requires us to go through each transaction, keep track <br/>
of each items count, convert the data into list of k_itemsets (defined above), sort it which is not necessary and return it.

In [7]:
from collections import defaultdict

def generate_c_1(transactions):
    c_1 = defaultdict(int)
    for trx in transactions:
        for item in trx:
            c_1[item] += 1
    c_1 = [k_itemset({ind}, support) for ind, support in c_1.items()]
    c_1.sort(reverse=True)
    return c_1

c_1 = generate_c_1(transactions)
c_1

[k_itemset(itemset={2}, support=3),
 k_itemset(itemset={1}, support=3),
 k_itemset(itemset={5}, support=3),
 k_itemset(itemset={4}, support=2),
 k_itemset(itemset={3}, support=1)]

=============================================================================================================================<br/>
Generating list of large itemsets of order k, L_k, from candiate list

In [8]:
def generate_l_k(c_k, min_support=1):
    return [c for c in c_k if min_support <= c.support]

L_1 = generate_l_k(c_1, min_support=2)
L_1

[k_itemset(itemset={2}, support=3),
 k_itemset(itemset={1}, support=3),
 k_itemset(itemset={5}, support=3),
 k_itemset(itemset={4}, support=2)]

To generate any 1<k candiate list, we check all combinations of k-1 large itemsets, and check if they <br/>
are different in only one element (each has one k-1 itemset object which the other does not), is so we join the two k-1 itemsets <br/>
and append to candiate list. Then for each set from first step we check if for all k-1 subsets of it is also large k-1, is it in L k-1.<br/>
If any of them is not large then it means that the entire subset is not large and we drop that subset from candiadates.

In [10]:
from itertools import combinations


def find_subsets_n(s, n):
    return [set(c) for c in combinations(s, n)]


def generate_c_k(L_k_1, k):
    c_k = []
    for l1, l2 in list(combinations(L_k_1, 2)):
        l1_ = l1.itemset
        l2_ = l2.itemset
        if len(l1_ ^ l2_)==2:
            l = l1_ | l2_
            c_k.append(l)
    c_k = list({k_itemset(c, 0) for c in c_k})
    for c in c_k:
        for subset in find_subsets_n(c.itemset, k-1):
            if subset not in L_k_1:
                c_k.remove(c)
                break
    return c_k
c_2 = generate_c_k(L_1, 2)
c_2

[k_itemset(itemset={2, 4}, support=0),
 k_itemset(itemset={1, 2}, support=0),
 k_itemset(itemset={1, 5}, support=0),
 k_itemset(itemset={1, 4}, support=0),
 k_itemset(itemset={4, 5}, support=0),
 k_itemset(itemset={2, 5}, support=0)]

=============================================================================================================================<br/>
Next we create C_bar_k which is the candiates list of size k which its support has been calculated and added or sorted.

In [12]:
def generate_c_bar_k(transactions, c_k):
    for t in transactions:
        c_t = [c for c in c_k if c.itemset.issubset(t)]
        for c in c_t:
            c.support += 1
    c_k.sort(reverse=True)
    return c_k
c_bar_2 = generate_c_bar_k(transactions, c_2)
c_bar_2

[k_itemset(itemset={1, 5}, support=3),
 k_itemset(itemset={2, 4}, support=2),
 k_itemset(itemset={1, 2}, support=2),
 k_itemset(itemset={2, 5}, support=2),
 k_itemset(itemset={1, 4}, support=1),
 k_itemset(itemset={4, 5}, support=1)]

=============================================================================================================================<br/>
We can put all these steps together in order to generate all the large itemsets

In [25]:
from collections import defaultdict
from itertools import combinations
from functools import total_ordering


@total_ordering
class k_itemset:
    def __init__(self, itemset: set, support=0):
        self.itemset = itemset
        self.support = support
    
    def __repr__(self):
        return f"k_itemset(itemset={self.itemset}, support={self.support})"
    
    def __eq__(self, other):
        try:
            return self.itemset == other.itemset
        except AttributeError:
            return self.itemset == other
    
    def __lt__(self, other):
        return self.support < other.support
    
    def __hash__(self):
        return hash(tuple(self.itemset))
    
    
def find_subsets_n(s, n):
    return [set(c) for c in combinations(s, n)]


def transform_transactions(transactions):
    items = set()
    for trx in transactions:
        trx = set(map(lambda t: t.lower(), trx))
        items = items.union(trx)
    items_map = {item: ind for ind, item in enumerate(sorted(items), 1)}
    trxs = [set(map(lambda t: items_map[t.lower()], trx)) for trx in transactions]
    return items_map, trxs


def generate_c_1(transactions):
    c_1 = defaultdict(int)
    for trx in transactions:
        for item in trx:
            c_1[item] += 1
    c_1 = [k_itemset({ind}, support) for ind, support in c_1.items()]
    c_1.sort(reverse=True)
    return c_1


def generate_c_k(L_k_1, k):
    c_k = []
    for l1, l2 in list(combinations(L_k_1, 2)):
        l1_ = l1.itemset
        l2_ = l2.itemset
        if len(l1_ ^ l2_)==2:
            l = l1_ | l2_
            c_k.append(l)
    c_k = list({k_itemset(c, 0) for c in c_k})
    for c in c_k:
        for subset in find_subsets_n(c.itemset, k-1):
            if subset not in L_k_1:
                c_k.remove(c)
                break
    return c_k


def generate_c_bar_k(transactions, c_k):
    for t in transactions:
        c_t = [c for c in c_k if c.itemset.issubset(t)]
        for c in c_t:
            c.support += 1
    c_k.sort(reverse=True)
    return c_k


def generate_l_k(c_k, min_support=1):
    return [c for c in c_k if min_support <= c.support]
        

# ==============================================================
transactions = [
    ["milk", "bread", "meat"],
    ["rice", "bread", "beer"],
    ["milk", "rice", "bread", "beer"],
    ["rice", "beer"]
]
min_support = 2

items_map, transactions = transform_transactions(transactions)
transactions = [
    [1, 3, 4],
    [2, 3, 5],
    [1, 2, 3, 5],
    [2, 5]
]
C_1 = generate_c_1(transactions)
L_1 = generate_l_k(C_1, min_support)

C = [[], C_1]
L = [[], L_1]

while (len(L[-1]) != 0):
    k = len(L) 
    c_k = generate_c_k(L[-1], k)
    c_k = generate_c_bar_k(transactions, c_k)
    if len(c_k) == 0:
        break
    C.append(c_k)
    L_k = generate_l_k(c_k, min_support=min_support)
    L.append(L_k)

# flattening L, and then sorting it
L = [itemset for l_k in L for itemset in l_k]
L.sort(reverse=True)

In [26]:
L

[k_itemset(itemset={3}, support=3),
 k_itemset(itemset={2}, support=3),
 k_itemset(itemset={5}, support=3),
 k_itemset(itemset={2, 5}, support=3),
 k_itemset(itemset={1}, support=2),
 k_itemset(itemset={2, 3}, support=2),
 k_itemset(itemset={1, 3}, support=2),
 k_itemset(itemset={3, 5}, support=2),
 k_itemset(itemset={2, 3, 5}, support=2)]

In [22]:
C[2]

[k_itemset(itemset={2, 5}, support=3),
 k_itemset(itemset={2, 3}, support=2),
 k_itemset(itemset={1, 3}, support=2),
 k_itemset(itemset={3, 5}, support=2),
 k_itemset(itemset={1, 2}, support=1),
 k_itemset(itemset={1, 5}, support=1)]

In [23]:
C[3]

[k_itemset(itemset={2, 3, 5}, support=2)]

In [24]:
C[4]

IndexError: list index out of range